In [5]:
from dotenv import load_dotenv, find_dotenv
import os
import redis

from langchain_community.embeddings import BedrockEmbeddings
from langchain_community.vectorstores.redis import Redis

embeddings = BedrockEmbeddings(model_id='cohere.embed-english-v3')

# read local .env file
_ = load_dotenv(find_dotenv())

redis_ip = value = os.getenv('REDIS_IP')
redis_port = value = os.getenv('REDIS_PORT')

# Connect to Redis
r = redis.Redis(host=redis_ip, port=redis_port)

# clear records
r.flushall()

info = r.info('keyspace')
info

{}

In [6]:
# seed vector db
seed_task = 'How many games did the Rays win in 2022?'

seed_plan = '''1. Import the schedule_and_record function from pybaseball:

```python
from pybaseball import schedule_and_record
```

2. Get the 2022 schedule and results for the Tampa Bay Rays:

```python
rays_2022 = schedule_and_record(2022, 'TB')
```

3. Filter the dataframe to games where the Rays won (W/L == 'W') or had a walk-off win (W/L == 'W-wo'):

```python
rays_wins = rays_2022[(rays_2022['W/L'] == 'W') | (rays_2022['W/L'] == 'W-wo')]
```

4. Count the number of rows (games won) in the filtered dataframe:

```python
num_wins = len(rays_wins)
print(f"The Tampa Bay Rays won {num_wins} games in 2022.")
```'''

seed_code = '''# 1. Import the schedule_and_record function from pybaseball
from pybaseball import schedule_and_record
# 2. Get the 2022 schedule and results for the Tampa Bay Rays
rays_2022 = schedule_and_record(2022, 'TB')
# 3. Filter the dataframe to games where the Rays won (W/L == 'W') or had a walk-off win (W/L == 'W-wo')
rays_wins = rays_2022[(rays_2022['W/L'] == 'W') | (rays_2022['W/L'] == 'W-wo')]
# 4. Count the number of rows (games won) in the filtered dataframe
num_wins = len(rays_wins)
print(f"The Tampa Bay Rays won {num_wins} games in 2022.")'''

metadata = [
    {
        "plan": seed_plan,
        "code": seed_code,
        "function_detail": "seed_function_detail"
    }
]
texts = ["seed_task"]

rds = Redis.from_texts(
    texts,
    embeddings,
    metadatas=metadata,
    redis_url=f"redis://{redis_ip}:{redis_port}",
    index_name="plans",
)

# write schema
rds.write_schema("redis_schema.yaml")

#rds.delete(ids=texts)

info = r.info('keyspace')
info

{'db0': {'keys': 1, 'expires': 0, 'avg_ttl': 0}}